# DLMR OpenLab \#2
-----------------

# Surrogate Text Representation 

You'll learn to:

*   Create Surrogate Text Representations using Pivots Permutations.
*   Index them using Elasticsearch as full-text search engine.



If you want to use a textual search engine (e.g. Elasticsearch, Apache Lucene, Whoosh) to search vectors, you can use **Surrogate Text Representations** (STR), a method that enables similarity search on top of standard textual search engines.

The key idea of STRs is to transform a real-valued vector into fake text that can be indexed and searched for using standard full-text search engines.

To do so, we need to **transform a real-valued vector $v \in \mathbb{R}^d$ into a vector of positive integers $\tilde{v} \in \mathbb{N}^{d'}$** representing *term frequencies* that can be mapped easily to text. This transformation $\tilde{v} = T(v)$ should have the following properies:

1.   $T$ should **preserve the rank** produced by the inner product between vectors; the more it does, the less precision we are sacrificing when scoring documents.

     $ q \cdot x < q \cdot y  \quad \Longrightarrow \quad T(q) \cdot T(x) < T(q) \cdot T(y)$ 

2.   $T$ should create **sparse** vectors; sparse vectors mean few terms per vector and thus shorter posting lists.


We will explore the **permutation of pivots** as the transformation function `T()`.

In [ ]:
import h5py
import numpy as np

But first, we'll get some data to index. We will use the GloVe word embedding dataset provided by [ann-benchmarks.com](https://github.com/erikbern/ann-benchmarks/#data-sets). It's a dataset of 1M+ word embeddings that also provides 10k queries for which true nearest neighbors are already computed. Vectors are compared with the cosine similarity.

In [ ]:
!wget --no-clobber http://ann-benchmarks.com/glove-100-angular.hdf5

The dataset is in HDF5 format, a popular scientific format that can be accessed using the `h5py` library. Let's see what's inside the datset file.

In [ ]:
data = h5py.File('glove-100-angular.hdf5', 'r')  # open in read mode
list(data.items())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

n_samples = 10_000
n_queries = 200
k = 100

# data can be accessed as numpy arrays
database = data['train'][:n_samples]
queries = data['test'][:n_queries]

true_scores = cosine_similarity(queries, database)
true_neighbors = true_scores.argsort(axis=1)[:, ::-1][:, :k] 

database[0]

Since we have the groundtruth provided by the dataset, we will compute the **Recall**, that is the fraction of true neighbors of a query that have been retrieved by the system, as metric of quality of the results by our indices.

$ \text{Recall} = \dfrac{|\text{True Neighbors} \cap \text{Retrieved Objects}|}{|\text{True Neighbors}|}\,.$

In [ ]:
def compute_recall(true_neighbors, predicted_neighbors):
  recalls = []
  for t, p in zip(true_neighbors, predicted_neighbors):
    intersection = np.intersect1d(t, p)
    recall = len(intersection) / len(t)
    recalls.append(recall)

  return np.mean(recalls)

## Pivots Permutation Representation

Let's select some pivots at random from the database:

In [ ]:
n_samples = len(database)
n_queries = len(queries)
n_pivots = 1000

pivots = np.random.choice(n_samples, n_pivots)
pivots = database[pivots]
pivots.shape

In [ ]:
xp_distances = cosine_similarity(database, pivots)
xp_distances

In [ ]:
# permutation: i -> pivot with rank i
xp_permutation = xp_distances.argsort(axis=1)
xp_permutation = xp_permutation[:, ::-1]  # reverse order, higher is better (cosine similarity)
xp_permutation

In [ ]:
# inverse permutation: i -> position of pivot i
xp_inv_perm = xp_permutation.argsort(axis=1)
xp_inv_perm

In [ ]:
# term frequency: i -> the importance of pivot i
xp_term_freq = n_pivots - xp_inv_perm
xp_term_freq

In [ ]:
# truncated term frequency: i -> the importance of pivot i
# but only keeping the nearest p pivots
prefix_length = 10
xp_trunc_tf = np.maximum(prefix_length - xp_inv_perm, 0)
xp_trunc_tf

In [ ]:
qp_distances = cosine_similarity(database, pivots)
qp_inv_perm = qp_distances.argsort(axis=1)[:, ::-1].argsort(axis=1)
qp_trunc_tf = np.maximum(prefix_length - qp_inv_perm, 0)

In [ ]:
def T(samples, pivots, prefix_length=None):
  """ Transform real-valued vectors into Term Frequencies-like vectors via
      Pivots Permutations. """
  n_pivots = len(pivots)
  prefix_length = prefix_length or n_pivots

  xp_distances = cosine_similarity(samples, pivots)
  xp_permutations = xp_distances.argsort(axis=1)[:, ::-1]
  xp_inv_permutat = xp_permutations.argsort(axis=1)
  xp_truncated_tf = np.maximum(prefix_length - xp_inv_permutat, 0)

  return xp_truncated_tf

In [ ]:
tx = T(database, pivots, prefix_length=250)
tq = T(queries, pivots, prefix_length=250)
tx.shape, tq.shape

Let's measure the **sparsity** of the transformed dataset. The sparsity of a vector is the fraction of zero elements in the vector.
The sparsity of a vector is directly related to the number of entries that will be stored in the posting lists of the textual search engine; therefore, the smaller the better.

In [ ]:
sparsity = 1 - (np.count_nonzero(tx) / tx.size)
sparsity

We can evaluate the performance loss introduced by the approximation by computing the inner product between the transformed vectors. This will be the operations that the textual search engine like Elasticsearch will internally perform.

In [ ]:
# compute scores (without an index, this is slow)

# this takes forever
# scores = tq.dot(tx.T)

# we use sparse multiplication (similar to what an index does)
# that is only efficient if sparsity is high enough
from scipy.sparse import csr_matrix
stq = csr_matrix(tq)
stx = csr_matrix(tx)

scores = stq.dot(stx.T).toarray()

# find 100 nearest neighbors
k = 100
sorted_indices = scores.argsort(axis=1)[:, ::-1]  # sort descending per row
topk = sorted_indices[:, :k]  # get **indices** of the topk images for each row

recall = compute_recall(true_neighbors, topk)
print(recall)

Once we have transformed our vectors, we can create surrogate text representations by repeating terms as many times as indicated by the transoformed vector, i.e., we interpret the transofmed vector as term frequencies.

In [ ]:
def surrogate_text(term_frequencies):
  """ Creates Surrogate Text by repeating the i-th term
      a number of time indicated by term_frequency[i].
  """
  tokens = []
  for i, tf in enumerate(term_frequencies):
    tokens += [f'f{i}'] * int(tf)

  text = ' '.join(tokens)
  return text

In [ ]:
surrogate_text(tx[0])

If the textual search engine supports boosting, we can generate shorter surrogate texts (that leads to lower query times):

In [ ]:
def surrogate_text_boost(term_frequencies):
  """ Creates Surrogate Text with Boosting.
      Instead of repeating a term N times, writes 'term^N'
      that in many full-text search engines has the same effect
      of setting the term frequency of that term to N.
      Useful to get shorter surrogate text representations.
  """
  tokens = []
  for i, tf in enumerate(term_frequencies):
    if tf:
      tokens.append(f'f{i}^{tf:g}')

  text = ' '.join(tokens)
  return text

In [ ]:
surrogate_text_boost(tq[0])

### Surrogate Text Representations on Elasticsearch

Let's see a working example of using STRs with Elasticsearch.
First, let's download and run an Elasticsearch instance.

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q --no-clobber
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0
!pip install -q elasticsearch==7.0.0

In [ ]:
# start server
import os
from subprocess import Popen, STDOUT, DEVNULL
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=DEVNULL, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# wait a bit for ES to start
!sleep 30

In [ ]:
! ps -ef | grep elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch(timeout=30)
print(es.ping())

Once Elasticsearch is up and running, let's create and index.
We need to specify one field (we name it `repr`) of type `text`, such that is searched with full-text semantics (TfIdf-like scoring). We also specify to use a white-space analyzer, that simply splits tokens on spaces.

In [ ]:
index_config = {
    "mappings": {
        "_source": {"enabled": False},  # do not store STR
        "properties": {"repr": {"type": "text"}}  # declare the field 'repr' as FULLTEXT
    },
    "settings": {
        "index": {"number_of_shards": 1, "number_of_replicas": 0},
        "analysis": {"analyzer": {"first": {"type": "whitespace"}}},  # tokenize by spaces, we don't need fancier analyzers
        # "similarity": {"inner_product": {"type": "scripted", "script": {"source": "return query.boost * doc.freq;"}}}  # multiply term frequencies only
    }
}

# delete any pre-existing indices
es.indices.delete('simsearch_index', ignore=(400, 404))

# create the index
es.indices.create('simsearch_index', index_config, ignore=400)

We use the utilities provided by the elasticsearch python package to bulk index documents.
We define a function that creates elasticsearch indexing commands and use `streaming_bulk` to process them sequentially.

In [ ]:
from elasticsearch.helpers import streaming_bulk
from tqdm.auto import tqdm

def generate_docs(index_name, v):
  for i, vi in enumerate(v):
    yield {'_index': index_name, '_id': i, 'repr': surrogate_text(vi)}

docs = generate_docs('simsearch_index', tx)
indexing = streaming_bulk(es, docs, chunk_size=150, max_chunk_bytes=2**26)
indexing = tqdm(indexing, total=n_samples)

for _ in indexing:
  pass

Let's check how many documents have been inserted.

In [ ]:
print(es.count(index='simsearch_index'))

Now, we can query the database. We specify we want to search the `repr` field. Since Elasticsearch supports the boosting syntax, we use the shorter surrogate text representation with boosting.

In [ ]:
qi = tq[0]  # first query

query = {
  "query": {"query_string": {"default_field": "repr", "query": surrogate_text_boost(qi)}},
  "from": 0, "size": k
}

results = es.search(index='simsearch_index', body=query)
results

Let's compute the recall for our query.

In [ ]:
topk = [int(hit['_id']) for hit in results['hits']['hits']]
topk = np.array([topk])  # make it a matrix with 1 row
topk

In [ ]:
qi_true_neighbors = true_neighbors[[0]]  # make it a matrix with 1 row
compute_recall(qi_true_neighbors, topk)

Next, let's perform the search for all our queries and compute the obtained recall.

#### **Exercise**: Measure the Mean Recall on Elasticsearch

*  Perform all the queries using Elasticsearch and measure the average query time and the average recall.
*  Try with different values of `prefix_length` and plot a *Recall vs Query Time* curve. What's the best recall for an average query time < 1 sec?
*  Try with different values of `n_pivots` and `prefix_length`. What's the best recall for an average query time < 1 sec? Optionally plot several *Recall vs Query Time* curves to summarize the results.


In [ ]:
D = []
I = []

for qi in tqdm(tq):
  query = {
    "query": {"query_string": {"default_field": "repr", "query": surrogate_text_boost(qi)}},
    "from": 0, "size": k
  }

  results = es.search(body=query, index='simsearch_index')

  Di = [hit['_score'] for hit in results['hits']['hits']]
  Ii = [int(hit['_id']) for hit in results['hits']['hits']]

  D.append(Di)
  I.append(Ii)

D = np.array(D)
I = np.array(I)

In [ ]:
recall = compute_recall(true_neighbors, I)
print(recall)

#### **(Advanced) Exercise**: Reordering with the Original Metric

* Use Elasticsearch and the Surrogate Text Representation approach to select a candidate set of results of size `k_s > k` for a query.
* Then reorder the `k_s` retrieved elements using the original features (`database` and `queries`) and the original metric (`cosine_similarity`) and keep the top `k` results as final result set.

What's the obtained average recall? and query time?

What's a good value of `k_s` (with respect to `k`) ?

## Additional Resources

*   https://github.com/erikbern/ann-benchmarks/
*   http://melisandre.deepfeatures.org/
*   https://www.elastic.co/elasticsearch/
